## 1. 关键词提取
互联网资源无穷无尽，如何获取到我们所需的那部分语料库呢？这需要我们给出特定的关键词，而基于问句的关键词提取上一节已经做了介绍，利用pynlpir库可以非常方便地实现关键词提取，比如：

In [1]:
import sys
import pynlpir

pynlpir.open()
s = '怎么才能把电脑里的垃圾文件删除'

key_words = pynlpir.get_key_words(s, weighted=True)
for key_word in key_words:
    print(key_word[0],'\t',key_word[1])
    

电脑 	 2.0
垃圾 	 2.0
文件 	 2.0
删除 	 1.0


## 2. 充分利用搜索引擎
有了关键词，想获取预料信息，还需要知道几大搜索引擎的调用接口，首先我们来探索一下百度，百度的接口是这样的：  
https://www.baidu.com/s?wd=机器学习 数据挖掘 信息检索  
把wd参数换成我们的关键词就可以拿到相应的结果，我们用程序来尝试一下：  
首先创建scrapy工程，执行：  

<font color=yellow>scrapy startproject baidu_search</font>

自动生成了baidu_search目录和下面的文件:  
<font color=yellow>scrapy genspider baidu_search1 baidu_search1.com</font>  
<br/> 
创建baidu_search/baidu_search/spiders/baidu_search.py文件，内容如下：

In [3]:
import scrapy
import sys

class BaiduSearch1Spider(scrapy.Spider):
    name = 'baidu_search1'
    allowed_domains = ['baidu.com']
    start_urls = ['http://www.baidu.com/s?wd=机器学习']

    def parse(self, response):
        print(response.body)


<font color=yellow>scrapy crawl baidu_search</font>

在settings.py中修改各项配置，如协议、用户代理、超时设置等

In [4]:
ROBOTSTXT_OBEY = False

USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_4) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'

DOWNLOAD_TIMEOUT = 5

再次执行：scrapy crawl baidu_search1  
<br/> 
可以看到大片大片的html了，现在将其写道临时文件中，修改parse()函数如下：

In [5]:
  def parse(self, response):
        filename = "result.html"
        with open(filename, 'wb') as f:
            f.write(response.body)

## 2. 语料提取
上面得到的仅是搜索结果，它只是一种索引，真正的内容需要进入到每一个链接才能拿到，下面我们尝试提取出每一个链接并继续抓取里面的内容，那么如何提取链接呢，我们来分析一下result.html这个抓取百度搜索结果文件  
<br/> 
我们可以看到，每一条链接都是嵌在class=c-container这个div里面的一个h3下的a标签的href属性  
修改parse()函数并添加如下代码：

In [ ]:
hrefs = response.selector.xpath('//div[contains(@class, "c-container")]/h3/a/@href').extract()
for href in hrefs:
    print href

In [ ]:
  def parse(self, response):
        hrefs = response.selector.xpath('//div[contains(@class, "c-container")]/h3/a/@href').extract()
        for href in hrefs:
            yield scrapy.Request(href, callback=self.parse_url)

    def parse_url(self, response):
        print len(response.body)

正常抓取后后将抓取下来的网页提取出正文并尽量去掉标签，如下：

In [1]:
import scrapy
import sys
from scrapy.utils.markup import remove_tags

class BaiduSearch1Spider(scrapy.Spider):
    name = 'baidu_search1'
    allowed_domains = ['baidu.com']
    start_urls = ['http://www.baidu.com/s?wd=机器学习']

    def parse(self, response):
        hrefs = response.selector.xpath('//div[contains(@class, "c-container")]/h3/a/@href').extract()
        containers = response.selector.xpath('//div[contains(@class, "c-container")]')
        
        for container in containers:
            href = container.xpath('h3/a/@href').extract()[0]
            title = remove_tags(container.xpath('h3/a').extract()[0])
            c_abstract = container.xpath('div/div/div[contains(@class, "c-abstract")]').extract()
            
            abstract = ""
            if len(c_abstract) > 0:
                abstract = remove_tags(c_abstract[0])
            
            request = scrapy.Request(href, callback=self.parse_url)
            request.meta['title'] = title
            request.meta['abstract'] = abstract
            yield request
            
            
    def parse_url(self, response):
        print("url:", response.url)
        print("title:", response.meta['title'])
        print("abstract:", response.meta['abstract'])
        content = remove_tags(response.selector.xpath('//body').extract()[0])
        print("content_len", len(content))

e:\python\lib\site-packages\ipykernel_launcher.py:3: ScrapyDeprecationWarning: Module `scrapy.utils.markup` is deprecated. Please import from `w3lib.html` instead.
  This is separate from the ipykernel package so we can avoid doing imports until


解释一下，首先我们在提取url的时候顺便把标题和摘要都提取出来，然后通过scrapy.Request的meta传递到处理函数parse_url中，这样在抓取完成之后也能接到这两个值，然后提取出content，这样我们想要的数据就完整了：url、title、abstract、content  
<br/> 
百度搜索数据几乎是整个互联网的镜像，所以你想要得到的答案，我们的语料库就是整个互联网，而我们完全借助于百度搜索引擎，不必提前存储任何资料，互联网真是伟大！